In [7]:
#=============================================================================
# XGBoost GPU 파이프라인 (cudf + cuml + xgboost GPU)
#=============================================================================
# 용도: train_sample_10pct.parquet (약 940MB) 전용
# 셀 구분: #%% 주석으로 노트북 셀 분리
# =============================================================================
#
# ⚠️ TODO: 과적합 해결 (Train ROC-AUC 0.89 vs Valid 0.73)
# ─────────────────────────────────────────────────────────
#
# [1] CFG.XGB_PARAMS에 정규화 강화 (Ridge/Lasso)
#     CFG.XGB_PARAMS['reg_lambda'] = 5.0    # Ridge (L2), 기본=1 → 5~10 시도
#     CFG.XGB_PARAMS['reg_alpha'] = 1.0     # Lasso (L1), 기본=0 → 0.5~2 시도
#
# [2] CFG.XGB_PARAMS에 scale_pos_weight 추가 (클래스 불균형 보정)
#     # 다운샘플링 후 1:1이면 불필요하지만, 원본 비율 반영 시:
#     CFG.XGB_PARAMS['scale_pos_weight'] = len(train_df[train_df['clicked']==0]) / len(train_df[train_df['clicked']==1])
#
# [3] CFG.XGB_PARAMS 트리 복잡도 줄이기
#     CFG.XGB_PARAMS['max_depth'] = 6       # 현재 8 → 5~6으로 줄이기
#     CFG.XGB_PARAMS['min_child_weight'] = 5 # 기본=1 → 5~10 (leaf 최소 샘플)
#     CFG.XGB_PARAMS['gamma'] = 0.3         # 기본=0 → 0.1~0.5 (split 최소 gain)
#
# [4] 피처 수 줄이기 (현재 1437개 → OHE 때문에 큼)
#     # interaction 피처 제거 테스트:
#     # add_interaction_features_gpu() 호출 주석 처리 후 재실행
#     # 또는 groupby 피처만으로 테스트
#
# ───────────────────────────────────────────────────────── -->

[PART 1] 라이브러리 임포트

In [8]:
import os
import ctypes

# Fix for OSError: libnvrtc.so.12: cannot open shared object file
# Portable solution: Load the library dynamically using ctypes
try:
    import nvidia.cuda_nvrtc
    # Handle namespace packages where __file__ might be None
    nvrtc_paths = nvidia.cuda_nvrtc.__path__
    nvrtc_lib = None
    
    for path in nvrtc_paths:
        candidate = os.path.join(path, 'lib', 'libnvrtc.so.12')
        if os.path.exists(candidate):
            nvrtc_lib = candidate
            break
            
    if nvrtc_lib:
        # RTLD_GLOBAL makes symbols available to subsequently loaded libraries (like cupy)
        ctypes.CDLL(nvrtc_lib, mode=ctypes.RTLD_GLOBAL)
        print(f"✅ [Fix] Successfully loaded: {nvrtc_lib}")
    else:
        print(f"⚠️ [Fix] Library file libnvrtc.so.12 not found in: {nvrtc_paths}")
except ImportError:
    print("⚠️ [Fix] 'nvidia.cuda_nvrtc' module not found. Please install it via pip or conda.")
except Exception as e:
    print(f"❌ [Fix] Failed to load libnvrtc.so.12: {e}")

import cudf
import cuml
import cupy as cp
import numpy as np
import pandas as pd
import xgboost as xgb
import os
import sys
import random
import gc
import datetime

from cuml.model_selection import train_test_split as gpu_train_test_split
from cuml.preprocessing import OneHotEncoder as cumlOneHotEncoder
from cupyx.scipy.sparse import csr_matrix as cp_csr_matrix, hstack as cp_hstack
from scipy.sparse import csr_matrix as sp_csr_matrix
from itertools import combinations
from sklearn.metrics import (
    roc_auc_score,
    average_precision_score,
    precision_recall_fscore_support,
    precision_recall_curve,
    auc,
    log_loss,
    accuracy_score
)
import warnings

warnings.filterwarnings('ignore')

# seq 임베딩 경로 추가
sys.path.append('/home/konkukstat/python3/HB/KSH/gpu_code')
sys.path.append('/home/konkukstat/python3/HB/KSH')

print(f"✅ cuDF {cudf.__version__}, cuML {cuml.__version__}")

✅ [Fix] Successfully loaded: /home/konkukstat/.local/lib/python3.10/site-packages/nvidia/cuda_nvrtc/lib/libnvrtc.so.12
✅ cuDF 25.12.00, cuML 25.12.00


[PART 2] 설정 및 유틸리티 함수

In [ ]:
class CFG:
    DATA_PATH = '/home/konkukstat/python3/workspace/data'
    TRAIN_FILE = 'train_sample_10pct.parquet'
    TEST_FILE = 'test.parquet'
    NOW = datetime.datetime.now().strftime('%Y%m%d_%H%M') 
    OUTPUT_PREFIX = f'XGB_GPU_{NOW}'
    OUTPUT_DIR = os.path.join('results', OUTPUT_PREFIX)

    SEEDS = [42, 106, 1031]
    NEG_RATIO = 1
    VALID_RATIO = 0.2

    XGB_PARAMS = {
        'objective': 'binary:logistic',
        'eval_metric': 'aucpr',
        'learning_rate': 0.08,
        'max_depth': 6,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'n_estimators': 500,
        'early_stopping_rounds': 30,
        'tree_method': 'hist',
        'min_child_weight': 3,
        'reg_lambda': 5,
        'device': 'cuda',
    }

    # Feature Engineering Flags (True: 사용 / False: 미사용)
    USE_SEQ_FEATURES = True
    USE_INTERACTION_FEATURES = True
    USE_GROUPBY_FEATURES = True
    USE_COUNT_FEATURES = True

    DEFAULT_AGG_DICT = {
        'history_a_1': ['mean', 'std'], 'history_a_2': ['mean', 'std'],
        'history_a_3': ['mean', 'std'], 'history_a_6': ['mean', 'std'],
        'feat_e_2': ['mean', 'std'], 'feat_e_3': ['mean', 'std'],
        'feat_c_8': ['mean', 'std'], 'feat_e_9': ['mean', 'std'],
        'feat_d_4': ['mean', 'std'],
        'l_feat_1': ['mean', 'std'], 'l_feat_2': ['mean', 'std'],
        'l_feat_5': ['mean', 'std'], 'l_feat_7': ['mean', 'std'],
        'l_feat_10': ['mean', 'std'], 'l_feat_15': ['mean', 'std'],
    }

    # 범주형 변수 목록 (unique.db 기반)
    CATEGORICAL_COLS = [
        'gender', 'age_group', 'inventory_id', 'day_of_week', 'hour',
        'l_feat_1', 'l_feat_2', 'l_feat_3', 'l_feat_4', 'l_feat_8',
        'l_feat_13', 'l_feat_16', 'l_feat_18', 'l_feat_19', 'l_feat_20',
        'l_feat_21', 'l_feat_22', 'l_feat_23', 'l_feat_24', 'l_feat_26',
        'l_feat_27', 'feat_e_4', 'feat_d_1', 'feat_d_2', 'feat_d_3',
        'feat_d_5', 'feat_d_6', 'feat_a_1', 'feat_a_2', 'feat_a_3',
        'feat_a_4', 'feat_a_6', 'feat_a_7', 'feat_a_8', 'feat_a_9',
        'feat_a_10', 'feat_a_11', 'feat_a_12', 'feat_a_13', 'feat_a_15',
        'feat_a_16', 'feat_a_17', 'feat_a_18'
    ]


def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)


def save_experiment_config(cfg):
    """설정값(파라미터, 플래그 등)을 JSON 파일로 저장"""
    import json
    config_data = {}
    
    # XGB_PARAMS 저장
    if hasattr(cfg, 'XGB_PARAMS'):
        config_data.update(cfg.XGB_PARAMS)
        
    # Feature Flags 및 기타 설정 저장
    for attr in dir(cfg):
        if attr.startswith('USE_') or attr in ['VALID_RATIO', 'NEG_RATIO', 'SEEDS', 'OUTPUT_PREFIX']:
            val = getattr(cfg, attr)
            # serialize numpy types
            if hasattr(val, 'item'): val = val.item()
            config_data[attr] = val
            
    def default(o):
        if hasattr(o, 'item'): return o.item()
        return str(o)
    
    filename = os.path.join(cfg.OUTPUT_DIR, f"{cfg.OUTPUT_PREFIX}_config.json")
    
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(config_data, f, indent=4, ensure_ascii=False, default=default)
        print(f"📄 Configuration saved: {filename}")
    except Exception as e:
        print(f"⚠️ Failed to save configuration: {e}")


[PART 3] 피처 엔지니어링 함수

In [10]:
try:
    _df_click_prob = pd.read_excel(os.path.join(CFG.DATA_PATH, 'high_click_numbers.xlsx'))
    CLICK_PROB_MAP = dict(zip(_df_click_prob['number'], _df_click_prob['click_prob']))
except FileNotFoundError:
    print("⚠️ 'high_click_numbers.xlsx' not found.")
    CLICK_PROB_MAP = {}

POS_LIST = frozenset({370, 528, 68, 561, 144, 227, 417, 442, 186, 395})
NEG_LIST = frozenset({154, 222, 84, 498, 434, 511, 216, 497, 309, 446})


def add_seq_features(df) -> None:
    """seq 컬럼에서 파생 피처 4개 생성 (cudf/pandas 호환)."""
    is_cudf = hasattr(df, 'to_pandas')
    seq_series = df['seq'].to_pandas() if is_cudf else df['seq']

    n = len(seq_series)
    seq_len = np.zeros(n, dtype=np.int32)
    avg_prob = np.zeros(n, dtype=np.float32)
    seq_neg = np.zeros(n, dtype=np.int32)
    seq_pos = np.zeros(n, dtype=np.int32)

    for i, s in enumerate(seq_series):
        if isinstance(s, str) and s:
            arr = [int(x) for x in s.split(",") if x]
            seq_len[i] = len(arr)
            if CLICK_PROB_MAP:
                probs = [CLICK_PROB_MAP.get(num, 0.0) for num in arr]
                avg_prob[i] = np.mean(probs) if probs else 0.0
            seq_neg[i] = sum(1 for x in arr if x in NEG_LIST)
            seq_pos[i] = sum(1 for x in arr if x in POS_LIST)

    if is_cudf:
        df['seq_len'] = cudf.Series(seq_len)
        df['avg_click_prob'] = cudf.Series(avg_prob)
        df['seq_neglogcount'] = cudf.Series(seq_neg)
        df['seq_poslogcount'] = cudf.Series(seq_pos)
    else:
        df['seq_len'] = seq_len
        df['avg_click_prob'] = avg_prob
        df['seq_neglogcount'] = seq_neg
        df['seq_poslogcount'] = seq_pos


def add_interaction_features_gpu(df, cols=None):
    """cudf DataFrame에 interaction 피처 추가"""
    if cols is None:
        cols = ['gender', 'age_group', 'inventory_id', 'day_of_week', 'hour']
    cols = [c for c in cols if c in df.columns]
    for col1, col2 in combinations(cols, 2):
        new_col = f'{col1}_{col2}'
        df[new_col] = df[col1].astype(str).str.cat(df[col2].astype(str), sep='_')


def add_groupby_features_gpu(train_df, val_df, test_df, feature_name, agg_dict=None):
    """cudf groupby 피처 (train에서만 통계 계산). 비수치형 컬럼은 자동 스킵."""
    if agg_dict is None:
        agg_dict = CFG.DEFAULT_AGG_DICT

    if feature_name not in train_df.columns:
        return

    # 존재하는 컬럼 + 수치형 컬럼만 필터링
    filtered_agg = {}
    for k, v in agg_dict.items():
        if k in train_df.columns:
            # cudf는 dtype.kind로 체크
            if train_df[k].dtype.kind in ['i', 'f', 'u']:  # int, float, unsigned
                filtered_agg[k] = v
    
    if not filtered_agg:
        return

    agg_stats = train_df.groupby(feature_name).agg(filtered_agg)
    new_cols = [f"{feature_name}_{col[0]}_{col[1]}" for col in agg_stats.columns]
    agg_stats.columns = new_cols
    agg_stats = agg_stats.reset_index()

    for target_df in [train_df, val_df, test_df]:
        merged = target_df.merge(agg_stats, on=feature_name, how='left')
        for col in new_cols:
            target_df[col] = merged[col]

    for col in new_cols:
        fill_val = float(train_df[col].mean())
        val_df[col] = val_df[col].fillna(fill_val)
        test_df[col] = test_df[col].fillna(fill_val)


def add_count_features_gpu(train_df, val_df, test_df, count_cols):
    """cudf Count Encoding: train에서만 빈도 계산"""
    for col in count_cols:
        if col not in train_df.columns:
            continue
        count_map = train_df[col].value_counts()
        train_df[f"{col}_count"] = train_df[col].map(count_map).fillna(0).astype('int32')
        val_df[f"{col}_count"] = val_df[col].map(count_map).fillna(0).astype('int32')
        test_df[f"{col}_count"] = test_df[col].map(count_map).fillna(0).astype('int32')


def gpu_hstack(matrices):
    """
    GPU용 자동 hstack 함수.
    cuDF DataFrame, CuPy array, cupyx sparse 등 무엇이든 합쳐줌.
    """
    converted_list = []
    for mat in matrices:
        if hasattr(mat, 'tocsr'):
            converted_list.append(mat.tocsr())
        elif hasattr(mat, 'to_pandas'):  # cudf DataFrame
            arr = mat.values
            if hasattr(arr, '__cuda_array_interface__'):
                arr = cp.asarray(arr)
            if arr.ndim == 1:
                arr = arr.reshape(-1, 1)
            converted_list.append(cp_csr_matrix(arr))
        elif isinstance(mat, cp.ndarray):
            if mat.ndim == 1:
                mat = mat.reshape(-1, 1)
            converted_list.append(cp_csr_matrix(mat))
        elif isinstance(mat, np.ndarray):
            if mat.ndim == 1:
                mat = mat.reshape(-1, 1)
            converted_list.append(cp_csr_matrix(cp.asarray(mat)))
        else:
            converted_list.append(cp_csr_matrix(mat))
    return cp_hstack(converted_list).tocsr()


[PART 4] 평가 함수

In [11]:
def compute_toss_score(y_true, y_pred):
    """Toss 대회 스코어 (CPU numpy 기반)."""
    if hasattr(y_true, 'get'):
        y_true = y_true.get()
    if hasattr(y_pred, 'get'):
        y_pred = y_pred.get()
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)

    y_pred_clipped = np.clip(y_pred, 1e-15, 1 - 1e-15)
    N_1 = np.sum(y_true == 1)
    N_0 = np.sum(y_true == 0)
    if N_1 == 0 or N_0 == 0:
        return 0.5, 0.0, 0.0

    weights = np.where(y_true == 1, 0.5 / N_1, 0.5 / N_0)
    wll = log_loss(y_true, y_pred_clipped, sample_weight=weights)
    ap = average_precision_score(y_true, y_pred)
    score = 0.5 * ap + 0.5 * (1 / (1 + wll))
    return score, ap, wll


def evaluate_model(model, X_train, y_train, X_val, y_val, threshold=0.5):
    """모델 평가 → DataFrame 반환 (GPU→CPU 변환 포함)."""
    results = {}
    for name, X, y_true in [('Train', X_train, y_train), ('Valid', X_val, y_val)]:
        y_prob = model.predict_proba(X)[:, 1]
        y_pred_class = (y_prob >= threshold).astype(int)
        y_true_np = np.asarray(y_true)
        y_prob_np = np.asarray(y_prob)

        roc = roc_auc_score(y_true_np, y_prob_np)
        prec_curve, rec_curve, _ = precision_recall_curve(y_true_np, y_prob_np)
        pr_auc = auc(rec_curve, prec_curve)
        avg_prec = average_precision_score(y_true_np, y_prob_np)
        ll = log_loss(y_true_np, y_prob_np)
        prec, rec, f1, _ = precision_recall_fscore_support(
            y_true_np, y_pred_class, average='binary', zero_division=0
        )
        acc = accuracy_score(y_true_np, y_pred_class)
        toss, _, _ = compute_toss_score(y_true_np, y_prob_np)

        results[name] = {
            'ROC-AUC': roc, 'PR-AUC': pr_auc, 'Avg Precision': avg_prec,
            'Precision': prec, 'Recall': rec, 'F1-Score': f1, 'Accuracy': acc,
            'Log Loss': ll, 'Toss Score': toss
        }
    return pd.DataFrame(results).T


def soft_voting_ensemble(prediction_files, output_file):
    """여러 시드의 예측 파일을 평균하여 앙상블"""
    if not prediction_files:
        print("❌ No prediction files.")
        return

    merged = pd.read_csv(prediction_files[0]).rename(columns={'clicked': 'click_0'})
    for i, f in enumerate(prediction_files[1:], 1):
        d = pd.read_csv(f).rename(columns={'clicked': f'click_{i}'})
        merged = merged.merge(d, on='ID')

    click_cols = [c for c in merged.columns if c.startswith('click_')]
    merged['clicked'] = merged[click_cols].mean(axis=1)

    merged[['ID', 'clicked']].to_csv(output_file, index=False)
    print(f"🎉 Ensemble saved: {output_file}")
    print(merged.head())


[PART 5] 메인 학습 루프

In [12]:
# ── 0. 출력 폴더 생성 및 설정 저장 ──
os.makedirs(CFG.OUTPUT_DIR, exist_ok=True)
save_experiment_config(CFG)

prediction_files = []

for seed in CFG.SEEDS:
    print(f"\n{'='*60}")
    print(f"🚀 Processing Seed: {seed}")
    print(f"{'='*60}")
    seed_everything(seed)

    # ── 1. 데이터 로드 (cudf) ──
    train_path = os.path.join(CFG.DATA_PATH, CFG.TRAIN_FILE)
    print(f"📂 Loading (GPU): {train_path}")
    full_train_gdf = cudf.read_parquet(train_path)
    print(f"✅ Train loaded: {full_train_gdf.shape}")

    try:
        test_gdf = cudf.read_parquet(os.path.join(CFG.DATA_PATH, CFG.TEST_FILE))
        print(f"✅ Test loaded: {test_gdf.shape}")
    except FileNotFoundError:
        print("⚠️ Test not found. Creating dummy.")
        test_gdf = full_train_gdf.head(100).drop(columns=['clicked'])
        test_gdf['ID'] = cudf.Series([f'TEST_{i}' for i in range(100)])

    # ── 1-1. 범주형 변수 dtype 변환 ──
    print("🔄 Converting categorical columns to str...")
    for col in CFG.CATEGORICAL_COLS:
        if col in full_train_gdf.columns:
            full_train_gdf[col] = full_train_gdf[col].astype(str)
        if col in test_gdf.columns:
            test_gdf[col] = test_gdf[col].astype(str)
    print("✅ Categorical dtype conversion done")

    # ── 2. seq 파생 피처 생성 ──
    if CFG.USE_SEQ_FEATURES:
        print("🛠️ Adding seq features...")
        add_seq_features(full_train_gdf)
        add_seq_features(test_gdf)
    else:
        print("⏭️ Skipping seq features...")

    # ── 3. seq 보관 + 제거 ──
    train_seq_raw = full_train_gdf['seq'].to_pandas() if 'seq' in full_train_gdf.columns else None
    test_seq_raw = test_gdf['seq'].to_pandas() if 'seq' in test_gdf.columns else None
    if 'seq' in full_train_gdf.columns:
        full_train_gdf.drop(columns=['seq'], inplace=True)
    if 'seq' in test_gdf.columns:
        test_gdf.drop(columns=['seq'], inplace=True)

    # ── 4. Train / Validation Split ──
    print("📊 Splitting train/validation...")
    train_gdf, val_gdf = gpu_train_test_split(
        full_train_gdf, test_size=CFG.VALID_RATIO,
        random_state=seed, stratify=full_train_gdf['clicked']
    )
    
    # seq도 같이 split (인덱스 활용)
    if train_seq_raw is not None:
        train_seq = train_seq_raw.loc[train_gdf.index.to_pandas()]
        val_seq = train_seq_raw.loc[val_gdf.index.to_pandas()]
        
    del full_train_gdf
    cp.get_default_memory_pool().free_all_blocks()
    gc.collect()

    # ── 5. Train set만 다운샘플링 ──
    print("🔻 Down-sampling train only...")
    clicked_1 = train_gdf[train_gdf['clicked'] == 1]
    clicked_0_all = train_gdf[train_gdf['clicked'] == 0]
    n_neg = min(int(len(clicked_1) * CFG.NEG_RATIO), len(clicked_0_all))
    clicked_0 = clicked_0_all.sample(n=n_neg, random_state=seed)
    
    # [FIX] 먼저 결합 및 셔플 (인덱스 유지)
    downsampled_gdf = cudf.concat([clicked_1, clicked_0]).sample(frac=1, random_state=seed)
    
    # seq 데이터 동기화 (같은 인덱스 사용) 및 Series 유지
    if train_seq_raw is not None:
        # train_seq 변수는 step 4에서 생성됨. 여기서는 downsampled_gdf의 인덱스로 필터링하고 리셋.
        # 인덱스는 원본 인덱스를 유지하고 있으므로 train_seq_raw(전체)에서 가져오거나 step 4의 train_seq에서 가져와도 됨.
        # CPU 코드와 동일하게 처리:
        train_seq = train_seq_raw.loc[downsampled_gdf.index.to_pandas()].reset_index(drop=True)

    # train_gdf 최종 확정
    train_gdf = downsampled_gdf.reset_index(drop=True)
    
    del clicked_1, clicked_0, clicked_0_all

    # ── 6. Feature Engineering ──
    print("🛠️ Feature Engineering...")

    if CFG.USE_INTERACTION_FEATURES:
        print("  -> Adding interaction features...")
        add_interaction_features_gpu(train_gdf)
        add_interaction_features_gpu(val_gdf)
        add_interaction_features_gpu(test_gdf)
    else:
        print("  -> Skipping interaction features...")

    if CFG.USE_GROUPBY_FEATURES:
        print("  -> Adding groupby features...")
        add_groupby_features_gpu(train_gdf, val_gdf, test_gdf, 'inventory_id')
    else:
        print("  -> Skipping groupby features...")
    
    if CFG.USE_COUNT_FEATURES:
        print("  -> Adding count features...")
        count_target_cols = ['gender', 'age_group', 'inventory_id', 'day_of_week', 'hour']
        count_target_cols = [c for c in count_target_cols if c in train_gdf.columns]
        add_count_features_gpu(train_gdf, val_gdf, test_gdf, count_target_cols)
    else:
        print("  -> Skipping count features...")

    # ── 7. 컬럼 분리 ──
    exclude_cols = {'clicked', 'ID'}
    all_cols = [c for c in train_gdf.columns if c not in exclude_cols]

    # 범주형 = CFG.CATEGORICAL_COLS + object dtype (interaction 피처 포함)
    cat_cols = [
        c for c in all_cols
        if (c in CFG.CATEGORICAL_COLS or train_gdf[c].dtype == 'object')
        and c in val_gdf.columns and c in test_gdf.columns
    ]
    # 수치형 = 나머지 (범주형 제외)
    num_cols = [c for c in all_cols if c not in cat_cols
                and c in val_gdf.columns and c in test_gdf.columns]

    # ── 8. 수치형 전처리 ──
    print("📋 Preprocessing numerical features...")
    # Scaling (cuml StandardScaler)
    from cuml.preprocessing import StandardScaler
    scaler = StandardScaler()
    train_gdf[num_cols] = scaler.fit_transform(train_gdf[num_cols])
    val_gdf[num_cols] = scaler.transform(val_gdf[num_cols])
    test_gdf[num_cols] = scaler.transform(test_gdf[num_cols])

    for col in num_cols:
        fill_val = float(train_gdf[col].mean())
        train_gdf[col] = train_gdf[col].fillna(fill_val)
        val_gdf[col] = val_gdf[col].fillna(fill_val)
        test_gdf[col] = test_gdf[col].fillna(fill_val)

    # ── 9. 범주형 전처리 (OneHotEncoder → CSR) ──
    print("📋 OneHot Encoding categorical features...")
    # cudf OneHotEncoder 또는 pandas로 변환 후 sklearn OneHotEncoder 사용
    # pandas로 변환하여 sklearn OHE 사용 (안정성)
    from sklearn.preprocessing import OneHotEncoder
    from sklearn.impute import SimpleImputer

    train_cat_pd = train_gdf[cat_cols].to_pandas()
    val_cat_pd = val_gdf[cat_cols].to_pandas()
    test_cat_pd = test_gdf[cat_cols].to_pandas()

    cat_imputer = SimpleImputer(strategy='most_frequent')
    train_cat_pd = pd.DataFrame(cat_imputer.fit_transform(train_cat_pd), columns=cat_cols)
    val_cat_pd = pd.DataFrame(cat_imputer.transform(val_cat_pd), columns=cat_cols)
    test_cat_pd = pd.DataFrame(cat_imputer.transform(test_cat_pd), columns=cat_cols)

    ohe = OneHotEncoder(sparse_output=True, handle_unknown='ignore')
    train_cat_sparse = ohe.fit_transform(train_cat_pd)
    val_cat_sparse = ohe.transform(val_cat_pd)
    test_cat_sparse = ohe.transform(test_cat_pd)
    print(f"  OHE shape: Train={train_cat_sparse.shape}")

    # ── 10. seq 임베딩 (선택 사항 — 사용하려면 아래 주석 해제) ──
    ### seq 임베딩 시작 ###
    # from src.models.seq_embedding_gpu import SeqEncoderGPU
    # seq_encoder = SeqEncoderGPU(embedding_dim=16)
    #
    # # Train: 학습 + 벡터 추출
    # seq_train = seq_encoder.get_features(
    #     train_seq, target_series=train_gdf['clicked'].to_pandas(),
    #     train_mode=True, epochs=5
    # )
    # # Valid/Test: 벡터만 추출
    # seq_val = seq_encoder.get_features(val_seq, train_mode=False)
    # seq_test = seq_encoder.get_features(test_seq_raw, train_mode=False)
    ### seq 임베딩 끝 → 아래 None을 삭제하고 위 주석 해제 ###
    seq_train = None
    seq_val = None
    seq_test = None

    # ── 11. Feature 결합 ──
    print("🔗 Combining features...")
    # GPU → pandas 변환 후 합치기 (XGBoost sklearn API 호환)
    from scipy.sparse import hstack as sp_hstack

    train_num_pd = train_gdf[num_cols].to_pandas()
    val_num_pd = val_gdf[num_cols].to_pandas()
    test_num_pd = test_gdf[num_cols].to_pandas()

    from scipy.sparse import csr_matrix as scipy_csr

    def pandas_hstack(matrices):
        converted = []
        for mat in matrices:
            if hasattr(mat, 'tocsr'):
                converted.append(mat.tocsr())
            elif hasattr(mat, 'values'):
                arr = mat.values
                if arr.ndim == 1:
                    arr = arr.reshape(-1, 1)
                converted.append(scipy_csr(arr))
            elif isinstance(mat, np.ndarray):
                if mat.ndim == 1:
                    mat = mat.reshape(-1, 1)
                converted.append(scipy_csr(mat))
            else:
                converted.append(scipy_csr(mat))
        return sp_hstack(converted).tocsr()

    hstack_train = [train_num_pd, train_cat_sparse]
    hstack_val = [val_num_pd, val_cat_sparse]
    hstack_test = [test_num_pd, test_cat_sparse]

    if seq_train is not None:
        hstack_train.append(seq_train)
        hstack_val.append(seq_val)
        hstack_test.append(seq_test)

    X_train = pandas_hstack(hstack_train)
    y_train = train_gdf['clicked'].to_pandas().values
    X_val = pandas_hstack(hstack_val)
    y_val = val_gdf['clicked'].to_pandas().values
    X_test = pandas_hstack(hstack_test)

    print(f"  X_train: {X_train.shape}, X_val: {X_val.shape}, X_test: {X_test.shape}")

    # ── 12. 모델 학습 (GPU) ──
    print("🔥 Training XGBoost (GPU)...")
    model = xgb.XGBClassifier(**CFG.XGB_PARAMS, random_state=seed)
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=50
    )

    # ── 13. 평가 ──
    print("📊 Evaluating...")
    eval_df = evaluate_model(model, X_train, y_train, X_val, y_val, threshold=0.5)
    print(eval_df.round(4).to_string())

    # ── 13-1. 평가 지표 CSV 저장 ──
    metrics_file = os.path.join(CFG.OUTPUT_DIR, f"{CFG.OUTPUT_PREFIX}_metrics_{seed}.csv")
    eval_df.to_csv(metrics_file)
    print(f"📊 Metrics saved: {metrics_file}")

    # ── 14. 예측 저장 ──
    test_pred = model.predict_proba(X_test)[:, 1]
    out_file = os.path.join(CFG.OUTPUT_DIR, f"{CFG.OUTPUT_PREFIX}_{seed}.csv")
    test_ids = test_gdf['ID'].to_pandas().values if 'ID' in test_gdf.columns else range(len(test_pred))
    sub = pd.DataFrame({'ID': test_ids, 'clicked': test_pred})
    sub.to_csv(out_file, index=False)
    prediction_files.append(out_file)
    print(f"💾 Saved: {out_file}")

    del train_gdf, val_gdf, test_gdf, X_train, X_val, X_test, model
    cp.get_default_memory_pool().free_all_blocks()
    gc.collect()

print(f"\n✅ All seeds processed: {prediction_files}")



🚀 Processing Seed: 42
📂 Loading (GPU): /home/konkukstat/python3/workspace/data/train_sampled_50pct.parquet


MemoryError: std::bad_alloc: out_of_memory: CUDA error (failed to allocate 22536303016 bytes) at: /tmp/pip-build-env-50yo9kd_/normal/lib/python3.10/site-packages/librmm/include/rmm/mr/cuda_memory_resource.hpp:51: cudaErrorMemoryAllocation out of memory

[PART 6] Metrics Summary

In [ ]:
print("\n📊 Creating metrics summary...")
all_metrics = []
for seed in CFG.SEEDS:
    metrics_file = os.path.join(CFG.OUTPUT_DIR, f"{CFG.OUTPUT_PREFIX}_metrics_{seed}.csv")
    if os.path.exists(metrics_file):
        df = pd.read_csv(metrics_file, index_col=0)
        df['Seed'] = seed
        all_metrics.append(df)

if all_metrics:
    summary_metrics = pd.concat(all_metrics, ignore_index=False)
    summary_file = os.path.join(CFG.OUTPUT_DIR, f"{CFG.OUTPUT_PREFIX}_metrics_summary.csv")
    summary_metrics.to_csv(summary_file)
    print(f"📊 Summary metrics saved: {summary_file}")
    print("\n" + summary_metrics.round(4).to_string())

#%% [PART 7] Soft Voting 앙상블
final_output = os.path.join(CFG.OUTPUT_DIR, f"{CFG.OUTPUT_PREFIX}_SoftVoting.csv")
soft_voting_ensemble(prediction_files, final_output)

print(f"\n{'='*60}")
print(f"🏁 모든 학습 및 앙상블 완료!")
print(f"  시드: {CFG.SEEDS}")
print(f"  개별 예측: {prediction_files}")
print(f"  앙상블 결과: {final_output}")
print(f"{'='*60}")



📊 Creating metrics summary...
📊 Summary metrics saved: XGB_GPU_metrics_summary.csv

       ROC-AUC  PR-AUC  Avg Precision  Precision  Recall  F1-Score  Accuracy  Log Loss  Toss Score  Seed
Train   0.8024  0.8112         0.8112     0.7341  0.7064    0.7200    0.7253    0.5538      0.7274    42
Valid   0.5801  0.0283         0.0283     0.0190  0.8641    0.0371    0.1513    0.9885      0.3013    42
Train   0.7872  0.7961         0.7961     0.7182  0.6869    0.7022    0.7087    0.5665      0.7172   106
Valid   0.5752  0.0263         0.0264     0.0188  0.8577    0.0368    0.1508    1.0344      0.2976   106
Train   0.8170  0.8255         0.8255     0.7496  0.7077    0.7280    0.7356    0.5411      0.7372  1031
Valid   0.5898  0.0282         0.0282     0.0192  0.8718    0.0376    0.1549    0.9526      0.3036  1031
🎉 Ensemble saved: XGB_GPU_SoftVoting.csv
             ID   click_0   click_1   click_2   clicked
0  TEST_0000000  0.425402  0.518557  0.443019  0.462326
1  TEST_0000001  0.346332  